# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [74]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [75]:
# checking your current working directory, /home/workspace
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
 
# Create a for loop to create a list of files and collect each filepath
# root =>/home/workspace/event_data
# dirs =>[]
# files=> ['2018-11-27-events.csv',....] 
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob , '/home/workspace/event_data/2018-11-14-events.csv',
    file_path_list = glob.glob(os.path.join(root,'*'))
    

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [41]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) # skip header
        
         # extracting each data row one by one and append it        
        for line in csvreader: 
            full_data_rows_list.append(line) 
            
 
 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [76]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [77]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [78]:
# TO-DO: Create a Keyspace 
try:
 session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                )
except Exception as e:
    print(e)

#### Set Keyspace

In [79]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [60]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

try:
    query = " CREATE TABLE IF NOT EXISTS sparkifydb.songs ( artist text,\
    itemInSession int,\
    length float,\
    sessionId int,\
    song text,\
    PRIMARY KEY ( itemInSession, sessionId ) )"   
    session.execute(query)
    
except Exception as e:
    print(e)


                    

In [61]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = " INSERT INTO songs  ( artist, itemInSession, length, sessionId, song)  "
        query = query + " VALUES ( %s, %s, %s, %s, %s ) " 
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9] ))

#### Do a SELECT to verify that the data have been inserted into each table

In [62]:
 
try:
    query = " SELECT artist, song, length FROM songs WHERE sessionId=338 and itemInSession=4  "
    rows = session.execute(query)
    for row in rows:
        print((row[0], row[1], row[2]))

except Exception as e:
    print(e)
    


('Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [80]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
 
     

try:
    session.execute( "drop table if exists sparkifydb.users")

    query = " CREATE TABLE IF NOT EXISTS sparkifydb.users ( artist text,\
    firstName text,\
    lastName text,\
    song text,\
    itemInSession int,\
    sessionId int,\
    userId int,\
    PRIMARY KEY  ((userId, sessionId), itemInSession)) "   
    
    session.execute(query) 
    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader: 
            query = " INSERT INTO users  ( artist, firstName, itemInSession, lastName, sessionId, song, userId) \
                 VALUES ( %s, %s, %s, %s, %s, %s, %s ) " 
            session.execute(query, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10]) ))
    
except Exception as e:
    print(e)
    
                    

In [85]:


         

query = " SELECT artist, song, firstName , lastName ,itemInSession  FROM users WHERE userId=10 and sessionId=182   "
query = " SELECT userId, sessionId, song, firstName ,itemInSession  FROM users WHERE userId=88 and sessionId=744   "
rows = session.execute(query)
for row in rows:
        print((row[0], row[1], row[2], row[3], row[4]  ))
        
                    

(88, 744, 'The Perfect Space', 'Mohammad', 1)
(88, 744, 'Mother Mother', 'Mohammad', 2)
(88, 744, 'Canada', 'Mohammad', 3)
(88, 744, '(Nice Dream)', 'Mohammad', 5)
(88, 744, 'Magic', 'Mohammad', 6)
(88, 744, 'Make Me Better', 'Mohammad', 7)
(88, 744, 'Minimum', 'Mohammad', 8)
(88, 744, 'Mon Ange', 'Mohammad', 9)
(88, 744, 'Mr. Jones', 'Mohammad', 10)
(88, 744, 'Money To Burn', 'Mohammad', 11)
(88, 744, 'Prayers', 'Mohammad', 12)
(88, 744, 'Die Kunst der Fuge_ BWV 1080 (2007 Digital Remaster): Contrapunctus XVII - Inversus', 'Mohammad', 13)
(88, 744, 'Revolution', 'Mohammad', 14)
(88, 744, 'I Might Be Wrong', 'Mohammad', 15)
(88, 744, 'Proper Education', 'Mohammad', 16)
(88, 744, 'Christeen', 'Mohammad', 17)
(88, 744, 'TRAS', 'Mohammad', 18)
(88, 744, 'Possessed', 'Mohammad', 19)
(88, 744, 'Picture', 'Mohammad', 20)
(88, 744, 'Space Travel', 'Mohammad', 21)
(88, 744, 'One Time', 'Mohammad', 22)
(88, 744, "Nothin' On You [feat. Bruno Mars] (Album Version)", 'Mohammad', 23)
(88, 744, 'Cov

In [54]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


try:
    session.execute( "drop table if exists sparkifydb.listened")

    query = " CREATE TABLE IF NOT EXISTS sparkifydb.listened (  firstName text,\
    lastName text,\
    song text,\
    userId int,\
    PRIMARY KEY  ( song, userId ) )"   
    
    session.execute(query) 
    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader: 
            query = " INSERT INTO listened ( firstName, lastName, song, userId) \
                 VALUES ( %s, %s, %s, %s) " 
            session.execute(query, (line[1], line[4], line[9], int(line[10]) ))
    
except Exception as e:
    print(e)

In [56]:
query = " SELECT firstName, lastName , song FROM listened  where song= 'All Hands Against His Own' "
 
rows = session.execute(query)
for row in rows:
        print( row[0], row[1]  )
         
 

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [123]:
## TO-DO: Drop the table before closing out the sessions

In [59]:
queries = ["DROP TABLE IF EXISTS songs",
           "DROP TABLE IF EXISTS users",
           "DROP TABLE IF EXISTS listened"]


for query in queries:
    try:
        session.execute(query)
        print('...',query)
    except Exception as e:
        print(e)
 

... DROP TABLE IF EXISTS songs <cassandra.cluster.ResultSet object at 0x7f3ed9c1aa58>
... DROP TABLE IF EXISTS users <cassandra.cluster.ResultSet object at 0x7f3ed9c1aa20>
... DROP TABLE IF EXISTS listened <cassandra.cluster.ResultSet object at 0x7f3ed9c1ada0>


### Close the session and cluster connection¶

In [139]:
session.shutdown()
cluster.shutdown()